# Pipeline 2018-2019
Basado en `generacion_features`, esta notebook ejecuta un pipeline similar para generar los indicadores para una base nueva de 2018 y 2019.

In [1]:
import numpy as np
import pandas as pd

In [2]:
db_path = '../data/bases'
years = [2018, 2019]

# 1. Preprocesamiento

Antes de trabajar con las bases de datos, necesitamos llenar campos vacíos, transformar los tipos de datos, y reproyectar las tablas a un formato más eficiente.

## 1.1 Procedimientos

Utilizando `src/utils/clean_data.py` tenemos que trabajar las bases de datos de [contratos de Compranet](https://sites.google.com/site/cnetuc/descargas) para limpiarlas, mezclarlas, y guardarlas en formato parquet. Vas a necesitar también `Directorio de unidades compradoras` y `Códigos repetidos en la tabla de procedimientos` de [imco/IRC#datos-procesados](https://github.com/imco/IRC/#datos-procesados).

In [4]:
from utils.clean_data import ensamblar_procedimientos

In [162]:
files = [
    'Contratos2018.xlsx',
    'Contratos2019_200525070452.xlsx'
]

In [3]:
%%time
procs = ensamblar_procedimientos(files, years, db_path)

(383903, 53)
(383903, 46)
CPU times: user 8.47 s, sys: 1.47 s, total: 9.94 s
Wall time: 2min 3s


In [4]:
procs.groupby('FECHA_ARCHIVO').size()

FECHA_ARCHIVO
2018    194191
2019    189654
dtype: int64

In [5]:
%%time
procs.to_csv(f'{db_path}/procedimientos-v2018.psv', sep='|', encoding='utf-8', index=False, quoting=1)
procs.to_parquet(f'{db_path}/procedimientos-v2018.parquet', engine='pyarrow')

CPU times: user 9.12 s, sys: 311 ms, total: 9.43 s
Wall time: 9.71 s


In [298]:
!ls -lh ../data/bases/proc*v2018* | cut -d' ' -f5-

 46M Jun 10 07:52 ../data/bases/procedimientos-v2018.parquet
293M Jun 10 07:52 ../data/bases/procedimientos-v2018.psv


## 1.2 Scraper

Similar a la [Tabla limpia del scraper](https://s3-us-west-2.amazonaws.com/opi-compranet/public/data/tabla_scraper_features.csv) obtenida de la [sección Datos Procesados en imco/irc](https://github.com/imco/irc#datos-procesados) para el periodo 2012-2017, buscamos generar una tabla con la siguiente iteración del scraper para 2018 y 2019. Las bases las hemos publicado en s3://ircimco/compranet-anuncios-2018.tar.gz y s3://ircimco/compranet-anuncios-2019.tar.gz.

In [106]:
!tar xf $db_path/compranet-anuncios-2018.tar.gz -C $db_path
!tar xf $db_path/compranet-anuncios-2019.tar.gz -C $db_path

In [107]:
!ls -lh $db_path/*anuncios* | cut -d' ' -f5-

787M Mar 29 17:38 ../data/bases/compranet-anuncios-2018.json
 58M Mar 29 17:44 ../data/bases/compranet-anuncios-2018.tar.gz
639M Mar 29 17:41 ../data/bases/compranet-anuncios-2019.json
 50M Mar 29 17:45 ../data/bases/compranet-anuncios-2019.tar.gz
725K Jun 11 20:13 ../data/bases/compranet-anuncios-2019.tsv


Cada base pesa alrededor de 500MB en formato json-lines. Para procesarlas de manera eficiente usaremos dask.

In [2]:
from dask.distributed import Client, progress
client = Client(threads_per_worker=1,
                n_workers=4,
                memory_limit='2GB')
client

Client Scheduler: tcp://127.0.0.1:44409 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 8.00 GB


In [3]:
%%time

import dask.bag as db
import json

df_anuncios_18 = (db.read_text('../data/bases/compranet-anuncios-2018.json', blocksize='64MB')
                 .map(json.loads)
                 .to_dataframe()
                 .compute())
df_anuncios_19 = (db.read_text('../data/bases/compranet-anuncios-2019.json', blocksize='64MB')
                 .map(json.loads)
                 .to_dataframe()
                 .compute())

print(df_anuncios_18.shape)
print(df_anuncios_19.shape)

(153766, 25)
(127023, 25)
CPU times: user 38.4 s, sys: 6.57 s, total: 45 s
Wall time: 43.1 s


In [8]:
assert list(df_anuncios_18.columns) == list(df_anuncios_19.columns)

In [10]:
df_anuncios_18['FECHA_ARCHIVO'] = 2018
df_anuncios_19['FECHA_ARCHIVO'] = 2019

In [11]:
anuncios = pd.concat([df_anuncios_18, df_anuncios_19])
del df_anuncios_18
del df_anuncios_19
anuncios.head()

,codigo,descripcion,referencia,tipo,categorias,descripcion_anuncio,notas,tipo_contratacion,entidad,fecha_publicacion,...,nombre_operador,email,web,tabla_datos,tabla_procedimientos,tabla_anexos,url,id_compranet,timestamp,FECHA_ARCHIVO
0,1522226,SERVICIO DE VIGILANCIA,CNF-CGGE-AD-DIR-NAY-2017-009,05. Adjudicación Directa LAASSP,3380-Servicios de vigilancia,SERVICIO DE VIGILANCIA PARA G.E. NAYARIT,Karla María Guzmán Sillas,Servicios,Nayarit,22/10/2017 03:43 PM,...,Karla María Guzmán Sillas,kguzman@conafor.gob.mx,,"{'rows': [{'visible': '', 'etiqueta': '', 'des...",{'rows': None},{'rows': None},https://compranet.hacienda.gob.mx/esop/toolkit...,1300262,1582563936,2018
1,1522243,SERVICIO DE MANTENIMIENTO Y CONSERVACIÓN DEL P...,CNF-CGGE-AD-DIR-NAY-2017-025,05. Adjudicación Directa LAASSP,3550-Reparación y mantenimiento de equipo de t...,SERVICIO DE “MANTENIMIENTO Y CONSERVACIÓN DEL ...,Karla María Guzmán Sillas,Servicios,Nayarit,22/10/2017 07:22 PM,...,Karla María Guzmán Sillas,kguzman@conafor.gob.mx,,"{'rows': [{'visible': '', 'etiqueta': '', 'des...",{'rows': None},{'rows': None},https://compranet.hacienda.gob.mx/esop/toolkit...,1300276,1582563967,2018
2,1522869,LICITACION MTTO AREAS VERDES Y EXTERIORES 2018,,01. Licitación Pública LAASSP,3510-Conservación y mantenimiento menor de inm...,LICITACION MTTO AREAS VERDES Y EXTERIORES 2018,ANGEL OLAZARAN GARZA,Servicios,Nuevo León,05/12/2017 07:48 PM,...,ANGEL OLAZARAN GARZA,angel.olazaran@imss.gob.mx,,"{'rows': [{'visible': '', 'etiqueta': '', 'des...",{'rows': None},"{'rows': [{'num': 0, 'archivo': '', 'url': '',...",https://compranet.hacienda.gob.mx/esop/toolkit...,1300789,1582565324,2018
3,1523166,PAVIMENTACION CON CONCRETO HIDRAULICO DE LA CA...,MHO/062-2017,06. Adjudicación Directa LOPSRM,6150-Construcción de vías de comunicación,LIMPIA TRAZO Y NIVELACION DE AREAS A URBANIZAR...,Alan Perez Aguilar,Obra Pública,Puebla,23/10/2017 01:32 PM,...,Alan Perez Aguilar,dobraspublicas@huauchinango.gob.mx,,"{'rows': [{'visible': '', 'etiqueta': '', 'des...",{'rows': None},{'rows': None},https://compranet.hacienda.gob.mx/esop/toolkit...,1301005,1582566030,2018
4,1523184,CONTRATACION DE MANTENIMIENTO VEHICULAR PARA E...,,03. Invitación a Cuando Menos Tres Personas LA...,3550-Reparación y mantenimiento de equipo de t...,CONTRATACION DE MANTENIMIENTO VEHICULAR PARA E...,Lilian María de los Ángeles Cortés Espinoza,Servicios,Morelos,02/11/2017 01:34 PM,...,Lilian María de los Ángeles Cortés Espinoza,lilian.cortes@imss.gob.mx,,"{'rows': [{'visible': '', 'etiqueta': '', 'des...",{'rows': None},"{'rows': [{'num': 0, 'archivo': '', 'url': '',...",https://compranet.hacienda.gob.mx/esop/toolkit...,1301021,1582566065,2018


In [14]:
anuncios['FECHA_ARCHIVO'].value_counts()

2018    153766
2019    127023
Name: FECHA_ARCHIVO, dtype: int64

### Campos objeto

Después de procesar la base de datos, tenemos que trabajar las columnas `tabla_datos` y `tabla_anexos` ya que contienen una lista de campos ingresados en la plataforma de compranet que están codificados como objetos. Estos son necesarios para el cálculo de indicadores y por eso vamos a hacerlos columna de un nuevo _dataframe_ y luego unirlo con nuestro conjunto original.

#### Tabla de datos

In [15]:
ejemplo_tabla_datos = anuncios[anuncios['FECHA_ARCHIVO'] == 2019]['tabla_datos'].iloc[1]

In [16]:
pd.DataFrame([f for f in ejemplo_tabla_datos['rows']])

,visible,etiqueta,descripcion,valor,url,archivo
0,,,,,,
1,,,,,,
2,Dato generado automáticamente y Visible al Pro...,\nNúmero del Procedimiento (Expediente),\n\nEste número se generará al momento de publ...,\nAA-019GYR994-E242-2017,,
3,Compilado por el Operador UC y Visible al Prov...,\nCarácter del procedimiento,\n\nIndicar el carácter del procedimiento\n,\nNacional,,
4,Compilado por el Operador UC y Visible al Prov...,\nMedio o forma del procedimiento,\n\nSeleccionar el medio o forma de participac...,\nElectrónica,,
5,Compilado por el Operador UC y Visible al Prov...,\nProcedimiento exclusivo para MIPYMES,\n\nDefina si se establece como requisito de p...,\nNo,,
6,,,,,,
7,,,,,,
8,Compilado por el Operador UC y Visible al Prov...,\nDatos relevantes de contrato,\n\nArchivo que contiene el informe con los da...,\n EcmReport.2149729.pdf (92 KB)DATOS RELEVAN...,/esop/toolkit/DownloadProxy/43397561?verify=13...,
9,Compilado por el Operador UC y Visible al Prov...,\nEscrito de justificación de la excepción a l...,\n\nArchivo que contiene el escrito de justifi...,\n (sin archivo adjunto),,


In [17]:
def tabla_datos_to_series(table):
    """
    Transforma la lista de campos en tabla_datos a una serie donde
    se usan los campos etiqueta y valor.
    """
    if not isinstance(table['rows'], list):
        return {}
    return {f['etiqueta'].strip(): f['valor'].strip() for f in table['rows'] if 'etiqueta' in f and len(f['etiqueta'])}

In [18]:
s = tabla_datos_to_series(ejemplo_tabla_datos)
assert s['Número del Procedimiento (Expediente)'] == 'AA-019GYR994-E242-2017'
assert s['Procedimiento exclusivo para MIPYMES'] == 'No'
pd.Series(s)

Número del Procedimiento (Expediente)                                                                          AA-019GYR994-E242-2017
Carácter del procedimiento                                                                                                   Nacional
Medio o forma del procedimiento                                                                                           Electrónica
Procedimiento exclusivo para MIPYMES                                                                                               No
Datos relevantes de contrato                                                        EcmReport.2149729.pdf  (92 KB)DATOS RELEVANTES...
Escrito de justificación de la excepción a la licitación pública, cuando aplique                                (sin archivo adjunto)
Testimonio del Testigo Social, si el procedimiento contó con el                                                 (sin archivo adjunto)
Documento de autorización de subcontratación                  

Con el método de transformación anterior lo aplicamos a todo el _dataframe_ de anuncios y generamos un nuevo _dataframe_ que concatenaremos más adelante. No estamos interesados en todas las columnas.

In [19]:
boolean_properties = {
    'Acta de fallo': 'archivo_fallo',
    'Acta de presentación y apertura de proposiciones': 'archivo_apertura',
    'Acta(s) de junta de aclaraciones': 'archivo_junta',
    'Convocatoria / Invitación': 'archivo_convocatoria',
    'Datos relevantes de contrato': 'archivo_contrato'
}

property_mapping = {
    'Número del Procedimiento (Expediente)': 'CODIGO_EXPEDIENTE',
    'Procedimiento exclusivo para MIPYMES': 'exclusivo_mipymes',
    'Testimonio del Testigo Social, si el procedimiento contó con el': 'testigo_social'    
}

property_mapping.update(boolean_properties)
original_columns = property_mapping.keys()

# Build a dataframe
tabla_datos = anuncios['tabla_datos'].apply(tabla_datos_to_series).to_list()
df_tabla_datos = pd.DataFrame(tabla_datos)[original_columns]

# Remap columns so they look like the previous scraper database
df_tabla_datos.rename(columns=property_mapping, inplace=True)

df_tabla_datos.head()

,CODIGO_EXPEDIENTE,exclusivo_mipymes,testigo_social,archivo_fallo,archivo_apertura,archivo_junta,archivo_convocatoria,archivo_contrato
0,AA-016RHQ001-E841-2017,No,(sin archivo adjunto),NaN,NaN,NaN,NaN,(sin archivo adjunto)
1,AA-016RHQ001-E855-2017,No,(sin archivo adjunto),NaN,NaN,NaN,NaN,(sin archivo adjunto)
2,LA-019GYR059-E225-2017,Sí,(sin archivo adjunto),(sin archivo adjunto),"E225 2017.pdf (1,385 KB)ACTA DE AP PPTAS",(sin archivo adjunto),CONVOCATORIA MTTO AREAS VERDES Y LIMP ... (...,LA CSR DAT REL.rar (94 KB)
3,AO-821071984-E7-2017,No,(sin archivo adjunto),NaN,NaN,NaN,NaN,(sin archivo adjunto)
4,IA-019GYR085-E44-2017,Sí,(sin archivo adjunto),"ACTA DE FALLO MANTTO. VEH. 2017.pdf (1,049 K...",ACTA PRESENTACIÓN Y APERTURA DE PROPOSIC... (...,ACTA JA SERV. MANTTO. VEH. 2018.pdf (504 KB)A...,CONVOCATORIOA I3P MANTENIMENTO VEHICULAR... (...,EcmReport.1650315 DATOS RELEVANTES DE CO... (...


Para limpiar la tabla, queremos dejar solo valores booleanos, i.e. 1 para la existencia de archivos, 0 de lo contrario. También podemos reemplazar etiquetas de datos faltantes por 0.

In [20]:
new_boolean_columns = boolean_properties.values()
for b in new_boolean_columns:
    c = df_tabla_datos.loc[:, b]
    c.loc[c.notnull()] = 1

df_tabla_datos.fillna(0, inplace=True)
df_tabla_datos.head()

,CODIGO_EXPEDIENTE,exclusivo_mipymes,testigo_social,archivo_fallo,archivo_apertura,archivo_junta,archivo_convocatoria,archivo_contrato
0,AA-016RHQ001-E841-2017,No,(sin archivo adjunto),0,0,0,0,1
1,AA-016RHQ001-E855-2017,No,(sin archivo adjunto),0,0,0,0,1
2,LA-019GYR059-E225-2017,Sí,(sin archivo adjunto),1,1,1,1,1
3,AO-821071984-E7-2017,No,(sin archivo adjunto),0,0,0,0,1
4,IA-019GYR085-E44-2017,Sí,(sin archivo adjunto),1,1,1,1,1


In [21]:
c = df_tabla_datos.loc[:, 'testigo_social']
c.replace([0, '(sin archivo adjunto)'], np.nan, inplace=True)
df_tabla_datos.loc[df_tabla_datos['testigo_social'].notnull(), 'testigo_social'] = 1
c.fillna(0, inplace=True)

c = df_tabla_datos.loc[:, 'exclusivo_mipymes']
c.replace('Sí', 1, inplace=True)
c.replace('No', 0, inplace=True)

c = df_tabla_datos.loc[:, 'CODIGO_EXPEDIENTE']
c.replace([0, 1], 'No registrado', inplace=True)

df_tabla_datos.head()

,CODIGO_EXPEDIENTE,exclusivo_mipymes,testigo_social,archivo_fallo,archivo_apertura,archivo_junta,archivo_convocatoria,archivo_contrato
0,AA-016RHQ001-E841-2017,0,0,0,0,0,0,1
1,AA-016RHQ001-E855-2017,0,0,0,0,0,0,1
2,LA-019GYR059-E225-2017,1,0,1,1,1,1,1
3,AO-821071984-E7-2017,0,0,0,0,0,0,1
4,IA-019GYR085-E44-2017,1,0,1,1,1,1,1


In [22]:
df_tabla_datos['testigo_social'].value_counts()

0                                                                                     280393
TESTIGO SOCIAL.pdf  (4,534 KB)TESTIGO SOCIAL                                               5
Testimonio Social.pdf  (24,078 KB)                                                         4
asignacion.pdf  (163 KB)                                                                   2
ACTA TESTIMONIO_IMSS-E63-AMDAD-2ene2019....  (483 KB)                                      2
                                                                                       ...  
FALLO E22.pdf  (131 KB)FALLO E22                                                           1
FALLO E9-2018.pdf  (122 KB)FALLO E9-2018                                                   1
FALLO E121.pdf  (122 KB)FALLO E121.pdf                                                     1
EcmReport.2070319 E56-2019 SEIP191204170...  (144 KB)DATOS RELEVANTES DEL CONTRATO         1
ACTA TESTIMONIAL E64-2018.pdf  (2,517 KB)                             

In [23]:
df_tabla_datos.shape

(280789, 8)

#### Tabla de anexos

In [25]:
ejemplo_tabla_anexos = anuncios[anuncios['FECHA_ARCHIVO'] == 2019]['tabla_anexos'].iloc[1]

In [26]:
# ejemplo de tabla_anexos para un registro
pd.DataFrame([f for f in ejemplo_tabla_anexos['rows']])

,num,archivo,url,descripcion,comentarios,fecha_modificacion
0,0,,,,,
1,1,"ACTA DE APERTURA E242.pdf (2,183 KB)",/esop/toolkit/DownloadProxy/43397228?verify=15...,ACTA DE APERTURA E242,,04/09/2019 06:28 PM
2,2,ACTA DE FALLO E242.pdf (665 KB),/esop/toolkit/DownloadProxy/43397322?verify=15...,ACTA DE FALLO E242,,04/09/2019 06:33 PM
3,3,INVITACIÓN MANTENIMIENTO DE ALUMINIO.doc (349...,/esop/toolkit/DownloadProxy/37585465?verify=15...,INVITACIÓN,,22/11/2017 08:58 PM


Dado que los campos son libres para los usuarios, tenemos una gran variedad de títulos para cada anexo. Usaremos expresiones regulares para encontrar patrones en estos campos, p. ej. si el anexo contiene la palabra "convenio" probablemente podamos asumir que es un convenio modificatorio. También aprovecharemos esta tabla para contabilizar el número de archivos subidos al expediente.

In [27]:
import re


re_convenio = re.compile('(convenio|modifica)', re.IGNORECASE)
assert re_convenio.search('DATOS RELEVANTES CONVENIO MODIFICATORIO BOLETOS DE AVION')
assert re_convenio.search('CONVENIO MODIFICATORIO')
assert re_convenio.search('CONVENIOS MODIFICATORIOS')
assert re_convenio.search('MODIFICACION')
assert re_convenio.search('MODIFICACION DE DATOS RELEVANTES POR CONVENIO')
assert re_convenio.search('CONVENIO')
assert re_convenio.search('AMPLIACION DE CONVENIO DE MONTO Y PLAZO ')
assert re_convenio.search('ACTA DE FALLO') == None
assert re_convenio.search('DATOS RELEVANTES DEL CONTRATO GNP inicial') == None
assert re_convenio.search('Acta Adjudicación Directa Nacional No. AA-019GYR020-E219-2018') == None
assert re_convenio.search('Acta Cierre Junta de Aclaraciones') == None
assert re_convenio.search('') == None

In [28]:
from collections import defaultdict


def tabla_anexos_to_series(table, keep_unknown_fields=False):
    """
    Transforma la lista de campos en tabla_anexos a una serie donde
    se usan los campos descripción para determinar la presencia de estos.
    Tratamos de replicar la base del scraper 2012-2017.
    """
    if not isinstance(table['rows'], list):
        return {}
    
    s = defaultdict(int)
    for row in table['rows']:
        if len(row['descripcion']) and len(row['archivo']):
            s['numero_archivos'] += 1
            if re_convenio.search(row['descripcion']):
                s['numero_convenios'] += 1
            elif keep_unknown_fields:
                s[row['descripcion']] = row['archivo']
    return s

In [29]:
s = tabla_anexos_to_series(ejemplo_tabla_anexos)
assert 'numero_convenios' not in s
assert s['numero_archivos'] == 3
pd.Series(s)

numero_archivos    3
dtype: int64

De igual manera aplicamos el transformador de la tabla a todo el _dataframe_ de anuncios, para generar un frame que mezclaremos con el principal.

In [32]:
tabla_anexos = anuncios['tabla_anexos'].apply(tabla_anexos_to_series).tolist()
df_tabla_anexos = pd.DataFrame(tabla_anexos)
df_tabla_anexos.fillna(0, inplace=True)
df_tabla_anexos.iloc[153767:153772]

,numero_archivos,numero_convenios
153767,3.0,0.0
153768,0.0,0.0
153769,1.0,1.0
153770,0.0,0.0
153771,7.0,0.0


**opcional**: en caso de que queramos revisar la variedad de entradas en tabla_anexos; podemos pasar `keep_unknown_fields=True` en `tabla_anexos_to_series` y correr la siguiente celda. El slice [:10] también se puede modificar para ver más resultados.

In [33]:
tabla_anexos_inspect = anuncios['tabla_anexos'].apply(lambda r: tabla_anexos_to_series(r, True)).tolist()
set(sum(map(lambda o: list(o.keys()), tabla_anexos_inspect[:10]), []))

{'1ER. J.A. LA-045-E207-2017',
 '2DA. ACTA DE J.A. LA-045-E207-2017',
 '3ER. Y ULTIMA ACTA J.A. LA-045-E207-2017',
 'ACTA DE FALLO MANTENIMIENTO VEHICULAR EJERCICIO 2018',
 'ACTA DE JUNTA DE ACLARACIONES',
 'ACTA FALLO LA-045-E207-2017',
 'ACTA JA DE SERVICIO DE MANTENIMIENTO VEHICULAR 2018',
 'ACTA PP LA-045-E207-2017',
 'ACTA PRESENTACIÓN Y APERTURA DE PROPOSICIONES SERV. MANTTO. VEH. 2018.',
 'ADQ. DE MATERIAL DE OSTEOSINTESIS Y ENDOPROTESIS PARA EJERCICIO 2018',
 'ANEXO DE CONVOCATORIA MTTO AREAS VERDES',
 'ANEXO DE CONVOCATORIA MTTO EXTERIORES',
 'Acta de Diferimiento ',
 'CONVOCATORIA E225 2017',
 'CONVOCATORIA SERV DE MANTENIMIENTO VEH 2018',
 'DIFERIMIENTO DE J.A. LA-045-E207-2017',
 'Dictamen Técnico Transportación',
 'INVESTIGAICON DE MERCADO',
 'JUNTA DE ACLARACIONES',
 'NUEVO FALLO SIST 56 POR RESOLUCION 5540 ',
 'PREGUNTAS Y RESP. LA-045-E207-2017',
 'RECTIFICACION ACTA FALLO LA-045-E207-2017',
 'REPORTE DE CONTRATO ',
 'Segunda Solicitud de Cotización',
 'Solicitud de Cot

In [34]:
df_tabla_anexos.shape

(280789, 2)

#### Mezclar tablas

Las tablas desagregadas de tabla_datos y tabla_anexos las vamos a insertar a nuestro _dataframe_ base.

In [36]:
print('anuncios', anuncios.shape)
print('tabla_datos', df_tabla_datos.shape)
print('tabla_anexos', df_tabla_anexos.shape)

anuncios (280789, 26)
tabla_datos (280789, 8)
tabla_anexos (280789, 2)


In [38]:
df_scraper = anuncios.join([df_tabla_datos, df_tabla_anexos])

# Ya no necesitaremos los campos tabla_, ya que los expandimos en nuevas columnas
df_scraper.drop(['tabla_datos', 'tabla_procedimientos', 'tabla_anexos'], axis=1, inplace=True)

df_scraper.head()

,codigo,descripcion,referencia,tipo,categorias,descripcion_anuncio,notas,tipo_contratacion,entidad,fecha_publicacion,...,CODIGO_EXPEDIENTE,exclusivo_mipymes,testigo_social,archivo_fallo,archivo_apertura,archivo_junta,archivo_convocatoria,archivo_contrato,numero_archivos,numero_convenios
0,1522226,SERVICIO DE VIGILANCIA,CNF-CGGE-AD-DIR-NAY-2017-009,05. Adjudicación Directa LAASSP,3380-Servicios de vigilancia,SERVICIO DE VIGILANCIA PARA G.E. NAYARIT,Karla María Guzmán Sillas,Servicios,Nayarit,22/10/2017 03:43 PM,...,AA-016RHQ001-E841-2017,0,0,0,0,0,0,1,0.0,0.0
0,1604036,"ADQUISICIÓN GRUPOS 010,040, Y CLAVE 2540 URGEN...",ADJ 1222 018 18,05. Adjudicación Directa LAASSP,2530-Medicinas y productos farmacéuticos,"ADQUISICIÓN GRUPOS 010,040, Y CLAVE 2540 URGEN...",JORGE LUIS PANTOJA PADILLA,Adquisiciones,Guanajuato,02/02/2018 05:53 PM,...,AA-016RHQ001-E841-2017,0,0,0,0,0,0,1,0.0,0.0
0,1627574,ADQUISICIÓN DE MATERIAL DE OSTEOSINTESIS,AA-019GYR091-E57-2018 / ADJ-DA-42-18,05. Adjudicación Directa LAASSP,"2540-Materiales, accesorios y suministros médicos",ADQUISICIÓN DE MATERIAL DE OSTEOSINTESIS,Jose Leobardo Flores Rodriguez,Adquisiciones,Puebla,01/03/2018 01:56 PM,...,AA-016RHQ001-E841-2017,0,0,0,0,0,0,1,0.0,0.0
0,1648823,ACEITE VEGETAL COMESTIBLE,143/33656,05. Adjudicación Directa LAASSP,2380-Mercancías adquiridas para su comercializ...,ACEITE VEGETAL COMESTIBLE,Abismael Reyes Lopez,Adquisiciones,Baja California Sur,27/03/2018 01:01 PM,...,AA-016RHQ001-E841-2017,0,0,0,0,0,0,1,0.0,0.0
0,1668556,Vocal de Ópera Ambulante,AD41-092-18,05. Adjudicación Directa LAASSP,"3390-Servicios profesionales, científicos y té...",Vocal de Ópera Ambulante,Brenda Nuñez Plancarte,Servicios,Baja California,04/05/2018 02:48 PM,...,AA-016RHQ001-E841-2017,0,0,0,0,0,0,1,0.0,0.0


In [40]:
del anuncios
del df_tabla_datos
del df_tabla_anexos

In [41]:
# Cool, ahora tenemos más columnas
df_scraper.shape

(280789, 33)

Probamos con un simple query el resultado final. Qué porcentaje registró Acta de Fallo?

In [42]:
cuenta = df_scraper["archivo_fallo"].sum() / df_scraper["archivo_fallo"].size
"Registros con Acta de fallo {:.0%}".format(cuenta)

'Registros con Acta de fallo 42%'

### Guardamos en disco

In [43]:
%%time
df_scraper.to_csv(f'{db_path}/scraper-v2018.csv', sep=',', encoding='utf-8', index=False, quoting=1)
df_scraper.to_parquet(f'{db_path}/scraper-v2018.parquet', engine='pyarrow')

TypeError: an integer is required (got type str)

In [44]:
!ls -lh ../data/bases/scraper*v201* | cut -d' ' -f5-

204M Jun 20 12:03 ../data/bases/scraper-v2018.csv


# 2. Cargar bases

In [294]:
df_procs = pd.read_parquet(f'{db_path}/procedimientos-v2018.parquet', engine='pyarrow')
df_procs.head()

,GOBIERNO,DEPENDENCIA,SIGLAS,NOMBRE_DE_LA_UC,CLAVEUC,RESPONSABLE,ESTRATIFICACION_MUC,FOLIO_RUPC,PROVEEDOR_CONTRATISTA,ESTATUS_EMPRESA,...,CLAVE_PROGRAMA,CUENTA_ADMINISTRADA_POR,ANUNCIO,ARCHIVADO,SIGLAS_PAIS,RAMO,ORGANISMO,C_EXTERNO,FECHA_ARCHIVO,IMPORTE_PESOS
0,APF,CENTRO NACIONAL DE CONTROL DEL GAS NATURAL,CENAGAS,CENAGAS-UNIDAD DE FINANZAS Y ADMINISTRACION DI...,018TON999,ADRIAN MERCADO ZEPEDA,NO MIPYME,367,EMERSON PROCESS MANAGEMENT,N/A,...,None,N/A,https://compranet.hacienda.gob.mx/esop/guest/g...,-1,MX,,None,None,2018,101629.0
1,APF,FONATUR MANTENIMIENTO TURISTICO SA DE CV,FONATUR MANTENIMIENTO,FONATUR MANTENIMIENTO-GERENCIA CENTRAL SUBDIRE...,021W3S002,FERNANDO PORFIRIO LORENZO VAZQUEZ,NO MIPYME,367,EMERSON PROCESS MANAGEMENT,N/A,...,None,N/A,https://compranet.hacienda.gob.mx/esop/guest/g...,-1,MX,,None,None,2018,730996.8
2,APF,INSTITUTO DE SEGURIDAD Y SERVICIOS SOCIALES DE...,ISSSTE,ISSSTE-DELEGACION ESTATAL DEL ISSSTE EN BAJA C...,051GYN052,REYNALDO PERAZA PERAZA,PEQUENA,485074,LUIS ERNESTO MEZA FLORES,N/A,...,None,N/A,https://compranet.hacienda.gob.mx/esop/guest/g...,-1,MX,,None,None,2018,272000.0
3,APF,INSTITUTO DE SEGURIDAD Y SERVICIOS SOCIALES DE...,ISSSTE,ISSSTE-DELEGACION ESTATAL DEL ISSSTE EN BAJA C...,051GYN052,REYNALDO PERAZA PERAZA,PEQUENA,485074,LUIS ERNESTO MEZA FLORES,N/A,...,None,N/A,https://compranet.hacienda.gob.mx/esop/guest/g...,-1,MX,,None,None,2018,269000.0
4,APF,INSTITUTO DE SEGURIDAD Y SERVICIOS SOCIALES DE...,ISSSTE,ISSSTE-JEFATURA DE SERVICIOS DE ADQUISICION DE...,051GYN007,ANTONIO RAMIREZ GUTIERREZ,NO MIPYME,None,LABORATORIOS DE BIOLOGICOS Y REACTIVOS DE MEXICO,N/A,...,None,N/A,https://compranet.hacienda.gob.mx/esop/guest/g...,-1,MX,,None,None,2018,64940400.0


In [296]:
df_scrap = pd.read_parquet(f'{db_path}/scraper-v2019.parquet', engine='pyarrow')
df_scrap.head()

,codigo,descripcion,referencia,tipo,categorias,descripcion_anuncio,notas,tipo_contratacion,entidad,fecha_publicacion,...,CODIGO_EXPEDIENTE,exclusivo_mipymes,testigo_social,archivo_fallo,archivo_apertura,archivo_junta,archivo_convocatoria,archivo_contrato,numero_archivos,numero_convenios
0,1463398,"SERVICIO DE IMPRESIÓN, FOTOCOPIADO Y DIGITALIZ...",ADJUDICACIÓN DIRECTA 41 III,05. Adjudicación Directa LAASSP,"3360-Servicios de apoyo administrativo, traduc...","SERVICIO DE IMPRESIÓN, DIGITALIZACIÓN Y FOTOCO...",Eglantina Rubio Alpizar,Servicios,México,22/11/2017 02:37 PM,...,AA-020VST001-E95-2017,0,0.0,0,0,0,0,1,0.0,0.0
0,1992326,"AROMATIZANTES,LIMPIADORES LÍQUIDOS, ÁCIDO MURI...",132/60066,05. Adjudicación Directa LAASSP,2380-Mercancías adquiridas para su comercializ...,"AROMATIZANTES,LIMPIADORES LÍQUIDOS, ÁCIDO MURI...",Irma Olga Arredondo Murillo,Adquisiciones,Yucatán,27/09/2019 11:03 AM,...,AA-020VST001-E95-2017,0,0.0,0,0,0,0,1,0.0,0.0
0,1843183,"REENCARPETADO 8 KMS POR CUERPO, DEL CAMINO LO...",LO-009000994-E2-2019,02. Licitación Pública LOPSRM,6150-Construcción de vías de comunicación,"REENCARPETADO 8 KMS POR CUERPO, DEL CAMINO LO...",Juan Luis Arredondo Duarte,Obra Pública,Sinaloa,15/01/2019 10:30 AM,...,AA-020VST001-E95-2017,0,0.0,0,0,0,0,1,0.0,0.0
0,1906272,ATUN ENLATADO,102/44002,05. Adjudicación Directa LAASSP,2380-Mercancías adquiridas para su comercializ...,ATUN ENLATADO,Jorge Luis Gallegos Montes,Adquisiciones,Coahuila,02/05/2019 10:35 AM,...,AA-020VST001-E95-2017,0,0.0,0,0,0,0,1,0.0,0.0
0,1890891,MATERIAL DE CURACIÓN,CONTRATO ISSSTE HRVGF LPI AD 352/2019,05. Adjudicación Directa LAASSP,"2540-Materiales, accesorios y suministros médicos",MATERIAL DE CURACIÓN,JUAN RAMON GARCIA LOPEZ,Adquisiciones,Jalisco,02/08/2019 07:14 PM,...,AA-020VST001-E95-2017,0,0.0,0,0,0,0,1,0.0,0.0
